In [13]:
# Requires GenePattern Notebook: pip install genepattern-notebook
import gp
import genepattern

# Username and password removed for security reasons.
genepattern.display(genepattern.session.register("https://cloud.genepattern.org/gp", "", ""))

GPAuthWidget()

# Reusable Notebook Cell Library

Below are a collection of self-contained cells that are useful for reuse in other notebooks.  Each is self-contained (i.e. you can cut and paste it as is within the GenePattern Notebook Environment with no additional dependencies).

When pasted into a new notebook you will need a GenePattern login cell to be present, and then you will need to 'Run' the cell to cause the UI to be displayed.

## Uploading Data to GenePattern

In [2]:
@genepattern.build_ui(name="Upload from Notebook server to GenePattern",collapse=False,
                      description="Upload a file from the Notebook to the GenePattern server. Displays a URL on the GenePattern server as output.", 
                      parameters={  "output_var":{"hide": True,},})
# upload the local file onto the GenePattern server so we can use it in the module
def uploadLocalNotebookFile(path_to_file_on_GPNB):
    filename = filenameAndPath.split("/")[1]
    uploaded = genepattern.get_session(0).upload_file(filename, path_to_file_on_GPNB)
    uploaded_url = uploaded.get_url()
    display(genepattern.GPUIOutput(files=[uploaded_url]))

UIBuilder(collapse=False, description='Upload a file from the Notebook to the GenePattern server. Displays a U…

In [3]:
import genepattern
import urllib
import os 
import re
import shutil
from IPython.core.display import HTML

def isUrl(anUrl):
    regex = re.compile(
        r'^(?:http|ftp)s?://' # http:// or https://
        r'(?:(?:[A-Z0-9](?:[A-Z0-9-]{0,61}[A-Z0-9])?\.)+(?:[A-Z]{2,6}\.?|[A-Z0-9-]{2,}\.?)|' #domain...
        r'localhost|' #localhost...
        r'\d{1,3}\.\d{1,3}\.\d{1,3}\.\d{1,3})' # ...or ip
        r'(?::\d+)?' # optional port
        r'(?:/?|[/?]\S+)$', re.IGNORECASE)

    return (re.match(regex, anUrl) is not None )


def downloadLinkToGPNB(anURL, FileName):
    print('a')
    urllib.request.urlretrieve(anURL, FileName)
    return './'+FileName  # the link path

    
def downloadLinkToGenePatternServer(anURL, filename):
    
    downloadurl_task = gp.GPTask(genepattern.get_session(0), 'urn:lsid:broad.mit.edu:cancer.software.genepattern.module.analysis:00034')
    downloadurl_task.param_load()
    downloadurl_job_spec = downloadurl_task.make_job_spec()
    downloadurl_job_spec.set_parameter("URL", anURL)
    downloadurl_job_spec.set_parameter("output.file", filename)
    job = genepattern.get_session(0).run_job(downloadurl_job_spec)
    output_list = job.get_output_files()  

    for file in output_list:     # Loop through each output file
        return file.get_url()
        

def uploadFileToGenePatternServer(File_or_URL, FileName):
    uploaded = genepattern.get_session(0).upload_file(FileName, File_or_URL) 
    my_local_url = uploaded.get_url()  
    return my_local_url

        
        
        
#@genepattern.build_ui( name="Upload file",
#                      description="Upload a file or URL to the GenePattern server or the GPNB server", 
#                parameters={
#               "File_or_URL": {
#                    "type": "file"
#                },
#                "FileName": {
#                    "type": "String"
#                },
#                "Where_To": {
#                    "type": "choice",
#                    "choices" : {"GPNB":"GPNB", "GPServer":"GP"}
#                },
#                "output_var": {
#                    "hide": True
#                }
#})
def _uploadFileAndGetLocalUrl(File_or_URL, FileName, Where_To):
    link = None
    
    if isUrl(File_or_URL):
        if (Where_To == "GPNB"):
            print("Save URL to GPNB " )
            link = downloadLinkToGPNB(File_or_URL, FileName)
        
        else:
            print("Save URL to GenePattern ")
            link = downloadLinkToGenePatternServer(File_or_URL, FileName)
    
    else:
        # its a file, ui builder should already have uploaded it locally
        if (Where_To == "GPNB"):
            print("Save file: all done its here in the current dir " )
            link = './'+FileName
        else:
            print("Save file to GenePattern ")
            link = uploadFileToGenePatternServer(File_or_URL, FileName)
    
    # now save it back as a new file local to the Notebook server
    # with open(output_gct_filename, 'w') as f:
    #    f.writelines('#1.2\n{}\t{}\n'.format(df2.shape[0], df2.shape[1] - 2))
    #    df2.to_csv(f, sep='\t', index= False)

    # upload the local file onto the GenePattern server so we can use it in the next module
    #
    # return "my_local_url"
    #return HTML('<a target="_blank" href="'+link+'">'+FileName+'</a>')
    #return HTML('<div><a class="gp-widget-job-output-file" data-kind="html" href="'+ link +'" onclick="return false;" data-toggle="popover" data-original-title="" title="" aria-describedby="popover798797">'+FileName+' <i class="fa fa-info-circle" style="color: rgb(128, 128, 128);"></i></a><div class="popover fade right in" role="tooltip" id="popover798797" style="top: -83.392px; left: 85.6165px; display: none;"><div class="arrow" style="top: 64.6865%;"></div><h3 class="popover-title" style="display: none;"></h3><div class="popover-content"><div class="list-group"><label>'+FileName+'</label><a class="list-group-item" href="https://genepattern.broadinstitute.org/gp/jobResults/1664268/feat.html?download" target="_blank">Download File</a><a class="list-group-item" href="'+ link +'" target="_blank">Open in New Tab</a><a class="list-group-item gp-widget-job-send-code" href="#">Send to Code</a><div><label style="padding-top: 10px;">Send to Existing GenePattern Cell</label><select class="form-control gp-widget-job-existing-task" style="margin-left: 0px;"><option>----</option></select></div><div style="display: none;"><label style="padding-top: 10px;">Send to New GenePattern Cell</label><select class="form-control gp-widget-job-new-task" style="margin-left: 0px;"><option>----</option></select></div></div></div></div></div>')   
    return genepattern.GPUIOutput(files=[link])

def printAndFlush(str):
    print(str)
    sys.stdout.flush()


def DownloadJobResultFile(Job_File_Url, File_Name):
    if os.path.isfile(File_Name):
        print("file already exists!")
        return

    # extract job number and file name from url
    job_num = Job_File_Url.split("/")[-2]
    remote_file_name = Job_File_Url.split("/")[-1]
    
    # get the job based on the job number passed as the second argument
    job = gp.GPJob(genepattern.get_session(0), job_num)

    # fetch a specific file from the job
    remote_file = job.get_file(remote_file_name)
    
    printAndFlush("downloading file...")
    response = remote_file.open()
    CHUNK = 16 * 1024
    with open(File_Name, 'wb') as f:
        while True:
            chunk = response.read(CHUNK)
            if not chunk:
                break
            f.write(chunk)
    printAndFlush("done!")




uploadFileUI = genepattern.GPUIBuilder(_uploadFileAndGetLocalUrl, name="Upload file",
                      description="Upload a file or URL to the GenePattern server or the GPNB server", 
                parameters={
                "File_or_URL": {
                    "type": "file"
                },
                "FileName": {
                    "type": "String"
                },
                "Where_To": {
                    "type": "choice",
                    "choices" : {"GPNB":"GPNB", "GPServer":"GP"}
                },
                "output_var": {
                    "hide": True
                }
})

uploadFileUI

UIBuilder(description='Upload a file or URL to the GenePattern server or the GPNB server', function_import='nb…

## Displaying Data

In [4]:
import os
import requests
import genepattern
from IPython.display import IFrame
@genepattern.build_ui(name="Display PDF", parameters={
    "image": {
        "name": "PDF to display:",
        "description": "PDF file (typically named Rplots.pdf) from the Seurat.QC module",
        "type": "file",
        "kinds": ["pdf"]
    },
    "height":{"default":850, "hide":True},
    "width":{"default":850, "hide":True},
    "output_var": {
        "default":"output_var",
        "hide": True
    }
})
def displayPdf(image, height, width):
    job_widget = nbtools.UIOutput(status="Getting file from the GenePattern server...")
    display(job_widget)
    f = gp.GPFile(genepattern.session.get(0), image)
    basename=os.path.basename(image)
    resp = requests.get(image, headers={
        'Authorization': f.server_data.authorization_header(), 
        'User-Agent': 'GenePatternRest'})
    job_widget.status = 'Writing pdf file to your workspace. This may take a minute.'
    with open(basename, "wb") as f:
        f.write(resp.content)
    
    job_widget.status = basename+' successfully written to the same folder as this notebook!'
    display(IFrame(basename,width, height))
    return

UIBuilder(function_import='nbtools.tool(id="Display PDF", origin="Notebook").function_or_method', name='Displa…

In [5]:
from IPython.core.display import display, HTML, display_html
from IPython.display import Image

%matplotlib inline

displayImageDescription = "Display an image within a cell of the notebook."
displayImageParamDef = {"image":{  "name":"Image file URL", "type":"file", "kinds":["png", "jpg","gif"]}, 
                        "image_1":{"name":"First image file URL", "type":"file", "kinds":["png", "jpg","gif"]}, 
                        "image_2":{"name":"Second image file URL", "type":"file", "kinds":["png", "jpg","gif"]}, 
                        "height": {"type": "number", "optional": True}, 
                        "width": {"type":"number", "optional": True}, 
                        "output_var":{"hide":True }}

def displayImage(image, height, width):
    return Image(url= image, width=width, height=height)

genepattern.GPUIBuilder(displayImage, collapse=False,
                        name="Display Image",
                        description=displayImageDescription,
                        parameters=displayImageParamDef)

UIBuilder(collapse=False, description='Display an image within a cell of the notebook.', function_import='nbto…

In [6]:
from IPython.core.display import display, HTML, display_html
from IPython.display import Image

%matplotlib inline

displayImagePairDescription = "Display a pair of images side-by-side within a cell of the notebook."
displayImagePairParamDef = {"image":{  "name":"Image file URL", "type":"file", "kinds":["png", "jpg","gif"]}, 
                        "image_1":{"name":"First image file URL", "type":"file", "kinds":["png", "jpg","gif"]}, 
                        "image_2":{"name":"Second image file URL", "type":"file", "kinds":["png", "jpg","gif"]}, 
                        "height": {"type": "number", "optional": True}, 
                        "width": {"type":"number", "optional": True}, 
                        "output_var":{"hide":True }}

def displayImagePair(image_1, image_2, height, width):
  
    # display(Image(url= image_1, width=width, height=height), Image(url= image_2, width=width, height=height))
    # display(Image(url= image_2, width=width, height=height))
    tdstyle=style='vertical-align: bottom;background-color:white'
    html_str = "<table><tr><td style='"+tdstyle+"'><img src="+image_1+" height="+str(height)+" width="+str(width)+"></td><td style='"+tdstyle+"'><img src="+image_2+" height="+str(height)+" width="+str(width)+"/></td></tr></table>"
    display_html(html_str, raw=True)
    return None   

genepattern.GPUIBuilder(displayImagePair, collapse=False,
                        name="Display Image Pair",
                        description=displayImagePairDescription,
                        parameters=displayImagePairParamDef)

UIBuilder(collapse=False, description='Display a pair of images side-by-side within a cell of the notebook.', …

In [7]:
@genepattern.build_ui(name="List Column Names", collapse=False, description="Display the column names from a tsv file.", 
                      parameters={  "output_var":{"hide": True,},})
def listColumnNames(path_to_file_on_GPNB):
    df = pd.read_csv( path_to_file_on_GPNB, sep='\t', header=0, index_col=False)
    display(list(df.columns.values))

UIBuilder(collapse=False, description='Display the column names from a tsv file.', function_import='nbtools.to…

## Utility functions

In [8]:
import pandas as pd
import gp.data as gpdata

@genepattern.build_ui(name="Convert TSV File into GCT", collapse=False, description="Extracts data from TSV file into GCT format and upload gct to the server ", 
                      parameters={  "output_var":{"hide": True,},})
# upload the local file onto the GenePattern server so we can use it in the module
def convertTSVDataIntoGCT(input_filename, output_filename, index_column, description_column, values_column_suffix):
    # read TSV into a data frame
    df = pd.read_csv(input_filename, sep='\t', header=0, index_col=False)
    # drop the default numeric row index
    df.reset_index(drop=True, inplace=True)
    # rename the first 2 columns to match norman GenePattern conventions of Name and Description
    df.rename(columns={index_column: "Name", description_column: "Description"}, inplace=True)
    # Set the Name and description as indexes which a GCT file expects in a dataframe
    df.set_index(["Name", "Description"], inplace=True);
    
    df2 = df.loc[:, df.columns.str.endswith(values_column_suffix)]
    # Show the top of the file
    display(df2.head())
    # Now write the dataframe out as a gct formatted file
    gpdata.write_gct(df2, output_filename)
    print("Wrote "+ output_filename)
    

UIBuilder(collapse=False, description='Extracts data from TSV file into GCT format and upload gct to the serve…

In [9]:
import gp, gp.data

@genepattern.build_ui(name="Read GCT file into DataFrame",
                      description="Reads a gct file on the GPNB server into a Pandas Dataframe in the current notebook.",
                      collapse=False, parameters={
    "output_var": {
        "default": "aDataFrame",
        "hide": False,
    }
})
def readGctIntoDataFrame(filename):
    gpfile = gp.GPFile(None, filename)
    df = gp.data.GCT(gpfile.uri)
    display(df.head())
    return df

UIBuilder(collapse=False, description='Reads a gct file on the GPNB server into a Pandas Dataframe in the curr…

In [10]:
import urllib

@genepattern.build_ui(collapse=False, name="Read GCT URL into DataFrame",
    description="Reads a GCT file from a remote URL into a Pandas Dataframe in the current notebook.", parameters={
   
    "output_var": {
        "default": "aDataFrameFromURL",
        "hide": False,
    }
})
def readGct_FromURL(link):
    df = gp.data.GCT(link)
    display(df.head())
    print("Loaded")
    return df

UIBuilder(collapse=False, description='Reads a GCT file from a remote URL into a Pandas Dataframe in the curre…

In [11]:
@genepattern.build_ui(description="Strip version numbers from ensemble ids and uploads to the GenePattern server.", 
                      parameters={  "output_var":{"hide": True,},})
# strip the version # from the ensemble gene id and upload
# the local file onto the GenePattern server so we can use it in the module
def stripEnsembleVersionIds(filename, output_filename):
    df = gpdata.GCT(filename)
    df.reset_index(drop=False, inplace=True)
    df['Name'] = df['Name'].apply(lambda x: x.split(".")[0])
    df.set_index(["Name", "Description"], inplace=True);
    display(df.head())
    gpdata.write_gct(df, output_filename)
    
    # and finally upload it to the GenePattern server from the Notebook server and display the resulting link
    uploaded = genepattern.get_session(0).upload_file(output_filename, output_filename)
    uploaded_url = uploaded.get_url()
    display(genepattern.GPUIOutput(files=[uploaded_url]))

UIBuilder(description='Strip version numbers from ensemble ids and uploads to the GenePattern server.', functi…

In [12]:
import gp, gp.data

@genepattern.build_ui(collapse=False, parameters={
    "output_var": {
        "hide": True,
    }
})
def readFile(filename, number_of_lines_to_print):
    count = 0;
    for line in open(filename):
        count = count + 1
        if (count > number_of_lines_to_print):
            break
        print(line)
    

           

UIBuilder(collapse=False, function_import='nbtools.tool(id="readFile", origin="Notebook").function_or_method',…